In [1]:
!pip install librosa
!pip install torchmetrics

In [2]:
import librosa

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import random

from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import torch
import torchmetrics
import os

from sklearn.metrics import roc_auc_score

In [3]:
import warnings
warnings.filterwarnings('ignore')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
class Config:
    SR = 32000
    N_MFCC = 13
    # Dataset
    ROOT_FOLDER = r"C:\Users\KimDongyoung\Downloads\SW중심대학"
    N_CLASSES = 2
    BATCH_SIZE = 128
    N_EPOCHS = 10
    LR = 1e-4
    DROPOUT_RATE = 0.3
    # Others
    SEED = 42
    
CONFIG = Config()

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CONFIG.SEED) # Seed 고정

In [5]:
# Load your DataFrame
df = pd.read_csv(os.path.join(CONFIG.ROOT_FOLDER, 'train.csv'))
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CONFIG.SEED)

# Update the 'path' column to have the full path
train['path'] = train['path'].apply(lambda x: os.path.join(CONFIG.ROOT_FOLDER, x))
val['path'] = val['path'].apply(lambda x: os.path.join(CONFIG.ROOT_FOLDER, x))

# Ensure test paths are also updated
test = pd.read_csv(os.path.join(CONFIG.ROOT_FOLDER, 'test.csv'))
test['path'] = test['path'].apply(lambda x: os.path.join(CONFIG.ROOT_FOLDER, x))

In [6]:
def get_mfcc_feature(df, train_mode=True):
    features = []
    labels = []
    for _, row in tqdm(df.iterrows()):
        file_path = row['path']
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            continue
        # Load audio file using librosa
        y, sr = librosa.load(file_path, sr=CONFIG.SR)
        
        # Extract MFCC features using librosa
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CONFIG.N_MFCC)
        mfcc = np.mean(mfcc.T, axis=0)
        features.append(mfcc)

        if train_mode:
            label = row['label']
            label_vector = np.zeros(CONFIG.N_CLASSES, dtype=float)
            label_vector[0 if label == 'fake' else 1] = 1
            labels.append(label_vector)

    if train_mode:
        return features, labels
    return features

In [7]:
# Call the function with the updated paths
train_mfcc, train_labels = get_mfcc_feature(train, True)
val_mfcc, val_labels = get_mfcc_feature(val, True)

44350it [10:23, 71.18it/s]
11088it [03:05, 59.62it/s]


In [8]:
class CustomDataset(Dataset):
    def __init__(self, mfcc, label):
        self.mfcc = mfcc
        self.label = label

    def __len__(self):
        return len(self.mfcc)

    def __getitem__(self, index):
        if self.label is not None:
            return self.mfcc[index], self.label[index]
        return self.mfcc[index]

train_dataset = CustomDataset(train_mfcc, train_labels)
val_dataset = CustomDataset(val_mfcc, val_labels)

In [9]:
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=False
)

In [10]:
class MLP(nn.Module):
    def __init__(self, input_dim=CONFIG.N_MFCC, hidden_dim=256, output_dim=CONFIG.N_CLASSES, dropout_rate=CONFIG.DROPOUT_RATE):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)  # Add dropout
        x = self.relu(self.fc2(x))
        x = self.dropout(x)  # Add dropout
        x = self.fc3(x)
        x = torch.sigmoid(x)
        return x

In [14]:
def train(model, optimizer, train_loader, val_loader, device, patience=5):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_score = 0
    best_model = None
    early_stop_count = 0
    
    for epoch in range(1, CONFIG.N_EPOCHS+1):
        model.train()
        train_loss = []
        for features, labels in tqdm(iter(train_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)
            
            optimizer.zero_grad()
            
            output = model(features)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val AUC : [{_val_score:.5f}]')
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
            early_stop_count = 0  # Reset early stopping counter
        else:
            early_stop_count += 1
        
        if early_stop_count >= patience:
            print("Early stopping")
            break
    
    return best_model

In [15]:
def multiLabel_AUC(y_true, y_scores):
    auc_scores = []
    for i in range(y_true.shape[1]):
        auc = roc_auc_score(y_true[:, i], y_scores[:, i])
        auc_scores.append(auc)
    mean_auc_score = np.mean(auc_scores)
    return mean_auc_score
    
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss, all_labels, all_probs = [], [], []
    
    with torch.no_grad():
        for features, labels in tqdm(iter(val_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)
            
            probs = model(features)
            
            loss = criterion(probs, labels)

            val_loss.append(loss.item())

            all_labels.append(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())
        
        _val_loss = np.mean(val_loss)

        all_labels = np.concatenate(all_labels, axis=0)
        all_probs = np.concatenate(all_probs, axis=0)
        
        # Calculate AUC score
        auc_score = multiLabel_AUC(all_labels, all_probs)
    
    return _val_loss, auc_score

model = MLP()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CONFIG.LR)

infer_model = train(model, optimizer, train_loader, val_loader, device)

100%|█████████████████████████████████████████████████████████████████████████████████| 87/87 [00:00<00:00, 647.93it/s]


Epoch [1], Train Loss : [1.83482] Val Loss : [0.57354] Val AUC : [0.82329]


100%|█████████████████████████████████████████████████████████████████████████████████| 87/87 [00:00<00:00, 610.04it/s]


Epoch [2], Train Loss : [0.67904] Val Loss : [0.59290] Val AUC : [0.83005]


100%|█████████████████████████████████████████████████████████████████████████████████| 87/87 [00:00<00:00, 640.16it/s]


...

Epoch [8], Train Loss : [0.42000] Val Loss : [0.36865] Val AUC : [0.92766]


100%|█████████████████████████████████████████████████████████████████████████████████| 87/87 [00:00<00:00, 560.31it/s]


Epoch [9], Train Loss : [0.40330] Val Loss : [0.34534] Val AUC : [0.93220]


100%|█████████████████████████████████████████████████████████████████████████████████| 87/87 [00:00<00:00, 580.76it/s]


Epoch [10], Train Loss : [0.39058] Val Loss : [0.33657] Val AUC : [0.93518]


In [16]:
test_mfcc = get_mfcc_feature(test, False)
test_dataset = CustomDataset(test_mfcc, None)
test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=False
)

50000it [16:18, 51.08it/s]


In [17]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for features in tqdm(iter(test_loader)):
            features = features.float().to(device)
            
            probs = model(features)

            probs  = probs.cpu().detach().numpy()
            predictions += probs.tolist()
    return predictions

In [18]:
preds = inference(infer_model, test_loader, device)

100%|███████████████████████████████████████████████████████████████████████████████| 391/391 [00:00<00:00, 849.75it/s]


In [19]:
submit = pd.read_csv(os.path.join(CONFIG.ROOT_FOLDER, 'sample_submission.csv'))
submit.iloc[:, 1:] = preds
submit.head()

,id,fake,real
0,TEST_00000,0.458532,0.534064
1,TEST_00001,0.126282,0.875924
2,TEST_00002,0.340868,0.656731
3,TEST_00003,0.050330,0.950279
4,TEST_00004,0.390456,0.604165


In [20]:
submit.to_csv(os.path.join(CONFIG.ROOT_FOLDER, '0704_submit.csv'), index=False)